<a href="https://colab.research.google.com/github/LeeSuA/MachineLearning-Study/blob/main/Article_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 케라스에서 제공하는 로이터 데이터셋(뉴스)을 46개의 토픽으로 분류하는 예제
- 단일 레이블 다중 분류 예제
- 훈련샘플 : 8982개 , 테스트샘플 : 2246개
- 로이터 데이터셋은 토큰화, 정수 인코딩이 돼있는 상태
- 테스트 데이터를 직접 예측

In [4]:
from keras.datasets import reuters

# num_words는 데이터에서 가장 자주 등장하는 단어 1만 개로 제한
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
     num_words=10000)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [5]:
print(len(train_data))
print(len(test_data))

8982
2246


In [6]:
# 데이터 준비
# 데이터를 벡터로 변환
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data) # 훈련 데이터 벡터 변환
x_test = vectorize_sequences(test_data) # 테스트 데이터 벡터 변환

In [7]:
# one-hot encoding
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels) # 훈련 레이블 벡터 변환
one_hot_test_labels = to_one_hot(test_labels) # 테스트 레이블 벡터 변환

In [8]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [9]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

In [10]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [12]:
# 9번째 에포크 이후 과대적합이 확인되었으므로 모델을 9번의 에포크로 새로 훈련시킴
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

Epoch 1/9
16/16 [==============================] - 2s 92ms/step - loss: 3.1292 - accuracy: 0.4031 - val_loss: 1.7898 - val_accuracy: 0.6290
Epoch 2/9
16/16 [==============================] - 1s 56ms/step - loss: 1.5500 - accuracy: 0.6769 - val_loss: 1.3615 - val_accuracy: 0.6970
Epoch 3/9
16/16 [==============================] - 1s 54ms/step - loss: 1.1050 - accuracy: 0.7579 - val_loss: 1.1458 - val_accuracy: 0.7530
Epoch 4/9
16/16 [==============================] - 1s 53ms/step - loss: 0.8768 - accuracy: 0.8187 - val_loss: 1.0509 - val_accuracy: 0.7720
Epoch 5/9
16/16 [==============================] - 1s 57ms/step - loss: 0.7044 - accuracy: 0.8563 - val_loss: 0.9806 - val_accuracy: 0.7960
Epoch 6/9
16/16 [==============================] - 1s 57ms/step - loss: 0.5611 - accuracy: 0.8851 - val_loss: 0.9483 - val_accuracy: 0.7920
Epoch 7/9
16/16 [==============================] - 1s 53ms/step - loss: 0.4445 - accuracy: 0.9079 - val_loss: 0.9191 - val_accuracy: 0.8030
Epoch 8/9
16/16 [===

In [13]:
results

[0.9762753248214722, 0.7818343639373779]

In [14]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
for m in range(0, 10):
  decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in test_data[m]])
  print(m,":",decoded_newswire)

557056/550378 [==============================] - 0s 0us/step
0 : ? the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to ? growth and expansion plans for ? inc and ? inc over the next two years a and p said the acquisition of ? in august 1986 and ? in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt ? ? of west germany reuter 3
1 : ? philippine sugar production in the 1987 88 crop year ending august has been set at 1 6 mln tonnes up from a provisional 1 3 mln tonnes this year sugar regulatory administration ? chairman ? yulo said yulo told reuters a surve

In [15]:
predictions = model.predict(x_test)

In [19]:
for i in range(0,30):
  print(np.argmax(predictions[i]))

3
10
1
4
13
3
3
3
3
3
1
4
1
3
1
11
4
3
19
3
8
3
3
3
9
3
4
6
10
3


In [18]:
print(test_labels[0:30])

[ 3 10  1  4  4  3  3  3  3  3  5  4  1  3  1 11 23  3 19  3  8  3  3  3
  9  3  4  6 10  3]
